In [9]:
# load from postgresql
import pandas as pd
import geopandas as gpd
from sqlalchemy import create_engine

engine = create_engine('postgresql://localhost:5432/zoning')
prc = gpd.read_postgis('prc_hgt_assessor_neighbor', engine, geom_col='geometry')
prc

/Users/alex/Library/Caches/pypoetry/virtualenvs/zoning-Yy6J4NSr-py3.12/lib/python3.12/site-packages/pandas/io/sql.py:1725: SAWarning: Did not recognize type 'geometry' of column 'geometry'
  self.meta.reflect(bind=self.con, only=[table_name], views=True)


,blklot,geometry,gen_hght,stories,neighbor_stories
0,0002001,"MULTIPOLYGON (((-13627824.006 4552427.02, -136...",40,0.0,0.0
1,0005001,"MULTIPOLYGON (((-13627436.696 4552355.112, -13...",40,0.0,0.0
2,0006001,"MULTIPOLYGON (((-13627617.921 4552325.021, -13...",40,0.0,0.0
3,0007001,"MULTIPOLYGON (((-13627803.588 4552294.893, -13...",40,0.0,0.0
4,0010001,"MULTIPOLYGON (((-13627722.222 4552281.181, -13...",40,3.0,4.0
...,...,...,...,...,...
152133,VACSTHES1,"MULTIPOLYGON (((-13625473.95 4539044.649, -136...",40,NaN,2.0
152134,VACSTHES2,"MULTIPOLYGON (((-13625265.24 4539027.566, -136...",40,NaN,2.0
152135,VACSTJES1,"MULTIPOLYGON (((-13626893.26 4548226.37, -1362...",240,NaN,10.0
152136,VACSTMIN,"MULTIPOLYGON (((-13626052.588 4548817.631, -13...",340,NaN,6.0


In [10]:
FEET_PER_METER = 3.28084

prc.loc[prc['blklot'] == '3581123'].area * FEET_PER_METER**2

87327    3406.255765
dtype: float64

In [11]:
# literally based on my house; could probably make this much better
AREA_PER_UNIT_FT2 = 3406
# right??
FT_PER_FLOOR = 10

In [12]:
prc['area_ft2'] = prc.area * FEET_PER_METER**2
prc['nominal_capacity'] = (prc['area_ft2'] / AREA_PER_UNIT_FT2) * (prc['gen_hght'] / FT_PER_FLOOR)

In [13]:
# now sanity check against my house
prc.loc[prc['blklot'] == '3581123']

,blklot,geometry,gen_hght,stories,neighbor_stories,area_ft2,nominal_capacity
87327,3581123,"MULTIPOLYGON (((-13629143.256 4545763.368, -13...",40,3.0,3.0,3406.255765,4.0003


In [25]:
ADDITIONAL_HEIGHT_FROM_NEIGHBOR = 1.3

# prc['height_increase'] = prc['neighbor_stories'] * FT_PER_FLOOR * ADDITIONAL_HEIGHT_FROM_NEIGHBOR - prc['gen_hght']
prc['height_increase'] = prc.apply(lambda row: max(row['neighbor_stories'] * FT_PER_FLOOR * ADDITIONAL_HEIGHT_FROM_NEIGHBOR - row['gen_hght'], 0), axis=1)

In [31]:
prc['unused_height'] = prc['gen_hght'] - prc['stories'] * FT_PER_FLOOR

# downtown
prc.cx[-13627342:-13624954, 4550444:4549098].explore('height_increase')

# SOMA
# prc.cx[-13627400:-13625100, 4549000:4548000].explore('height_increase')


# castro
# -13629823.314186 4546555.162218
# -13628334.695595 4545499.870292
# prc.cx[-13629823:-13628334, 4546555:4545499].explore('height_increase')

In [30]:
# folium????

# import folium

# m = folium.Map(location=[37.7749, -122.4194], zoom_start=15)

# folium.Choropleth(
#     geo_data=prc[::100],
#     name='choropleth',
#     columns=['blklot', 'height_increase'],
#     fill_color='YlGn',
# ).add_to(m)

# m